In [79]:
import pandas as pd
import nltk

In [23]:
import json
data = {}
with open('homework.json') as f:
    i = 0
    for line in f:
        d = json.loads(line)
        data[i] = d
        i+=1


In [24]:
df = pd.DataFrame.from_dict(data, orient = 'index')
df = df.drop(columns = ['emotion_9']) #always encoded as 1. drop

In [25]:
df.head()

,emotion_0,emotion_1,emotion_2,emotion_3,emotion_4,emotion_5,emotion_6,emotion_7,emotion_8,headline,summary,worker_id
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Parenting Lessons From a Partial Eclipse,I don’t do partiality. Maybe it was finally ti...,95524929
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,On Catalonia,Catalan lawmakers declared independence from S...,30266977
2,0.0,0.0,-2,0.0,0.0,0.0,0.0,0.0,0.0,A Spark of Romance Under Fluorescent Lighting,"Rose Burlingham, an art dealer and writer, mar...",54197764
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,Apple Removes Apps From China Store That Help ...,The world’s most valuable company appears to h...,41177265
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Commerce Secretary’s Offshore Ties to Putin ‘C...,"Wilbur Ross, the commerce secretary, retained ...",56518865


In [26]:
# remove weird boolean rows
df['drop'] = False
for i in range(9):
    df[f'emotion_{i}'] = pd.to_numeric(df[f'emotion_{i}'],errors = 'coerce')
    df.loc[df[f'emotion_{i}'].isin([0.,1.])==False,"drop"] = True

# get rid of dumb text
drops = ["fnord","-1","cat","-2",""]
df.loc[df[f'headline'].isin(drops),"drop"] = True
df.loc[df[f'summary'].isin(drops),"drop"] = True
df = df.loc[df['drop']==False]

#sum ratings on story level
df2 = df.drop(columns = ["drop","worker_id"]).groupby(["headline","summary"]).sum().reset_index()
sc = df2['summary'].value_counts().reset_index()
summary_drops = list(sc.loc[sc["summary"]>1,"index"]) #only keep single summary-headline
df3 = df2.loc[~df2['summary'].isin(summary_drops)] # remove duplicates

#get modal rating for story
rename_dic = dict(zip([f"emotion_{x}" for x in range(9)],[x for x in range(9)]))
df3 = df3.rename(columns = rename_dic)
df3['emotion']=df3[[x for x in range(9)]].idxmax(axis=1)

In [75]:

print(summary_drops)

['Here’s what you need to know to start your day.', 'Here’s what you need to know at the end of the day.', 'Welcome to the latest edition of the Smarter Living newsletter.', 'Here’s what you need to know about the week’s top stories.', 'Recent residential sales in New York City and the region.', 'Suggested reading from critics and editors at The New York Times.', 'The Breakdown: conversation starters and context, drawn from the day’s news in Australia.', 'It’s a big city, with plenty to do, see, hear and watch. Here’s a sampling of cultural highlights this weekend and over the week ahead.', 'New movies and TV shows are added to streaming platforms all the time. Here are the ones we think are most interesting in the coming month.', 'Look closely at this image, stripped of its caption, and join the moderated conversation about what you and other students see.', 'This week’s properties include a five-bedroom in Briarcliff Manor, N.Y., and a four-bedroom in Englewood, N.J.']


In [106]:
df3 = df3.reset_index().rename(columns = {"emotion":"labels","index":"idx"})
print(df3.head())
df3[["idx","headline","summary","labels"]].to_csv("clean.csv",index=False)
df3[["idx","headline","summary","labels"]].loc[:1000,:].to_csv("clean_test.csv",index=False)

   idx  idx                                           headline  \
0    0    0  #metoo floods social media with stories of har...   
1    1    1  $1.3 million homes in oregon, michigan and cal...   
2    2    2                             $1.3 million for that?   
3    3    3  $1.5 million homes in california, montana and ...   
4    4    4  $1.5 million homes in rhode island, california...   

                                             summary    0    1    2    3    4  \
0  many women, as well as men, are using the hash...  0.0  0.0  0.0  1.0  4.0   
1  a dwell-style home in portland; a 1929 tudor h...  1.0  0.0  0.0  0.0  1.0   
2  whatever paul manafort may or may not have don...  0.0  0.0  0.0  0.0  0.0   
3  a floor-through condominium in san francisco, ...  2.0  0.0  0.0  0.0  0.0   
4  an 18th-century farmhouse outside newport, a h...  0.0  0.0  2.0  0.0  1.0   

     5    6    7    8  labels  
0  2.0  0.0  7.0  2.0       7  
1  3.0  0.0  0.0  5.0       8  
2  3.0  0.0  0.0  5.

In [29]:
df3["labels"].value_counts()

5    4242
8    1454
7    1256
0     530
1     501
4     417
3     413
2     319
6      48
Name: labels, dtype: int64

In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

for var in ["headline","summary"]:
    df3[var] = df3[var].apply(lambda x: ps.stem(x))
    
train = df3.loc[:2000]
test =  df3.loc[2000:]




ps = nltk.stem.PorterStemmer()

train_vectors = vectorizer.fit_transform(train["headline"])
test_vectors = vectorizer.transform(test["headline"])

In [108]:
import time
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, train['labels'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

In [109]:
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(test['labels'], prediction_linear, output_dict=True)

Training time: 0.427969s; Prediction time: 0.680982s


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [110]:
from sklearn.metrics import accuracy_score

results = accuracy_score(prediction_linear,test['labels'], normalize = True)
#results = f1_score(prediction_linear,test['labels'],average = 'accuracy')

In [111]:
X = vectorizer.fit_transform(train["headline"])

In [112]:
print(results)

0.48788300835654597


In [113]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score

In [114]:
import numpy as np

In [116]:
train, test = train_test_split(df3, test_size=0.2, random_state=1)
train.head()

,idx,idx,headline,summary,0,1,2,3,4,5,6,7,8,labels
7415,7415,7567,these n.b.a. dancers bring their high kicks co...,"with packed routines, the hard-working brookly...",0.0,0.0,4.0,0.0,0.0,3.0,1.0,0.0,4.0,2
925,925,944,"as anti-semitism rises, ‘i don’t feel safe in ...",acts of anti-semitism have increased nearly 10...,0.0,7.0,0.0,3.0,0.0,4.0,0.0,7.0,1.0,1
2973,2973,3019,health bill appears dead as pivotal g.o.p. sen...,senator susan collins of maine announced her o...,1.0,1.0,2.0,1.0,1.0,2.0,0.0,2.0,1.0,2
4850,4850,4942,n.f.l. picks: patriots over broncos; cowboys o...,"the los angeles rams, dallas cowboys and new e...",1.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0,3.0,5
227,227,239,a consumer advocate battles equifax and a storm,“i managed to respond to the first several hun...,0.0,0.0,0.0,1.0,1.0,7.0,0.0,2.0,0.0,5


In [124]:
import time

shuffled_i = list(range(len(df3)))
np.random.shuffle(shuffled_i)
train_size = int(len(df3)*.8)
train_obs = shuffled_i[:train_size]
test_obs = shuffled_i[train_size:]

train = df3.loc[train_obs,:]
test =  df3.loc[test_obs,:]

kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
np.random.seed(1)

t0 = time.time()
pipeline_svm = make_pipeline(vectorizer, 
                            SVC(probability=True, kernel="rbf", class_weight="balanced"))

grid_svm = GridSearchCV(pipeline_svm,
                    param_grid = {'svc__C': [0.01, 0.1, 1]}, 
                    cv = kfolds,
                    scoring = "accuracy",
                    verbose=1,   
                    n_jobs=-1)
t1 = time.time()
time_linear_train = t1-t0

grid_svm.fit(train["headline"], train["labels"])
grid_svm.score(test["headline"], test["labels"])
print(time_linear_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
0.0015668869018554688


In [125]:
def report_results(model, X, y):
    pred_proba = model.predict_proba(X)[:, 1]
    pred = model.predict(X)        

    #auc = roc_auc_score(y, pred_proba)
    acc = accuracy_score(y, pred)
    f1 = f1_score(y, pred,average = 'macro')
    prec = precision_score(y, pred,average = 'macro')
    rec = recall_score(y, pred,average = 'macro')
    result = { 'f1': f1, 'acc': acc, 'precision': prec, 'recall': rec}
    return result

In [126]:
print(report_results(grid_svm.best_estimator_, test["headline"], test["labels"]))

{'f1': 0.275997983054981, 'acc': 0.4591503267973856, 'precision': 0.28220906226803005, 'recall': 0.2790799602902322}
